In [3]:
from IPython.display import HTML, display

# Simulating Planetary Orbits with a Symplectic Integrator

The name of this library in Fluxions in homage to Isaac Newton, whose early name for differential calculus was "the method of fluxions."  (For an entertaining work of  fiction that places the invention of calculus in historical context, I highly recommend Neal Stephenson's Baroque Cycle.)  The problem Newton was trying to solve was calculating the motion of the planets around the sun under the influence of gravity.  He succeeded in calculating a highly accuracte approximation based on all the planets moving around the sun in elliptical orbits.  This approximation effectively treats the sun as a very heavy stationary body and all the planets orbit around it.  It ignores the gravitational forces between planets and any movement of the sun itself.  It's a good approximation because the sun is much heavier than the planets put toghether (approximately 99.8% of the of the mass of the solar system), but it's not perfect because Jupiter in particular is heavy enough to throw things off.

**Kepler's Laws for Elliptical Orbits**

<img src="../graphics/kepler.gif">

Citation: http://hyperphysics.phy-astr.gsu.edu/hbase/kepler.html

## Numerical Solution of Differential Equations with Symplectic Integrators
How do scientists and engineers today compute planetary orbits to the highest standards of precision suitable for launching space vehicles?  They use numerical integrators to solve the Newton's equations of motion.  In particular, they use a special class of numerical integrators called **symplectic integrators**.  Most students in STEM fields who encounter differential equations are likely to see a very unrepresentative sample of ones that are analytically tractable, such as a simple harmonic oscillator.  It turns out that most differential equations of interest do **not** have analytical solutions, but have to be solved numerically instead.  This is a mature field of study in applied mathematics, and many methods exist.

A simple numerical method is the highly intuitive Forward Euler Method, in which the equation is discretized in time:
<img src="../graphics/forward_euler_method.png">

Citation: https://en.wikipedia.org/wiki/Euler_method#/media/File:Euler_method.svg

As the picture suggests, discretization errors can gradually accumulate and the approximated trajectory can drift away from the true solution.  These errors can be mitigated by using higher order methods with small step size, but unless an integrator is carefully constructed, it is likely to have a drift where it either gains or loses energy as the system evolves.  This destroys its ability to make accurate calculations over a large number of steps.

### Hamilton's Equations
No discussion of symplectic integrators would be complete without introducing Hamilton's equations.  This is a formalism for solving the equations of motion in a physical system.  The spatial coordinates are $q = (q_1, ... q_n)$ and their "conjugate momenta" are $p = (p_1, ... p_n)$.  

$$\frac{d\mathcal{p}_i}{dt} = -\frac{\partial \mathcal{H}}{\partial q_i} \\
\frac{dq_i}{dt} = +\frac{\partial \mathcal{H}}{\partial p_i}$$

The highly symmetrical dual structure of these equations give Hamiltonian systems their special properties.
This can look like a bit much at first, but there's a physical intuition that helps to understand it:
* $q_i$ are the x, y, and z coordinates of different bodies in the problem
* $p_i$ are the x, y, and z coordinates of the momentum of the bodies; $p_i = m_i v_i$

For a classical mechanical system,
$$\mathcal{H} = T + U$$
where $T$ is the total kinetic energy of the system, and $U$ is the total potential energy of the system.
A very important special case is easier to solve: when the Hamiltonian is *separable* and *time invariant*.  
In this case, the kinetic energy $T = T(p)$ depends only on the momenta $p$, 
and the potential energy $U = U(q)$ depends only on the position.

It can be proven mathematically that any Hamiltonian system (i.e. a system that evolves according to these differential equations) has two special properties:
* It conserves volume in phase space, $dV = dp dq$
* It conserves energy $\mathcal{H}(p, q)$



### Bad Idea: Simulating a Hamiltonian System with a Non-Symplectic Integrator
What happens if you try to simulate a Hamiltonian system with a non-symplectic integrator? It loses its special properties.  The simulated solution will not conserve energy and volume in phase space.

<img src="../graphics/non_symplectic_ham_steps.png">

Citation: https://www.av8n.com/physics/symplectic-integrator.htm

### What is a Symplectic Integrator?
The figure above shows a typical behavior of a non-symplectic integrator that is gradually leaking energy.  If this were a planetary orbit, the simulation would show the planet crashing into the sun at a time when it should still enjoy a stable orbit.

A symplectic integrator on the other hand **respects** the two key symmetries of the Hamiltonian system.
* It conserves energy
* It conserves volume

### Conservation of Energy

<img src="../graphics/symplectic_ham_steps.png">

Citation: https://www.av8n.com/physics/symplectic-integrator.htm

### Convervation of Volume in Phase Space

Here are visualizations of the fact that symplectic integrators conserve volume in phase space, but non-symplectic integrators do not:
<img src="../graphics/non_symplectic_ham_volume.png">
<img src="../graphics/symplectic_ham_volume.png">

Citation: https://www.av8n.com/physics/symplectic-integrator.htm

One way to think of a symplectic integrator is that it models the behavior of another symplectic system (one that conserves energy and volume in phase space) that is very close to the true system.  
Whereas a non-symplectic integrator models a system close to the true system, but one that is not symplectic.

### Better Idea: Leapfrog Integration -- A Simple Symplectic Integrator
Fortunately there is a simple scheme for numerically solving separable Hamiltonian systems that is symplectic.  It is called Leapfrog Integration.  Here is a presentation in "traditional" coordinates $x$, $v$ and $a$ for position, velocity, and acceleration respectively.  Please note that velocities are indexed at half-integer time steps.
\begin{align}
x_i &= v_{i-1} + v_{i-1/2} \Delta t \\
a_i &= F(x_i) / m_i \\
v_{i+1/2} &= v_{i-1/2} + a_i \Delta t
\end{align}

Here is an equivalent version with only integer indices that is better suited to direct translation into computer code.
This version of the equations was used in the planetary simulation we wrote.
\begin{align}
x_{i+1} &= x_i + v_i \Delta t + \frac{1}{2} a_i \Delta t^2
\end{align}

citation: https://en.wikipedia.org/wiki/Leapfrog_integration

This embarrassingly simple code is at the heart of the planetary simulation presented below:
```python
# Perform leapfrog integration simulation
# https://en.wikipedia.org/wiki/Leapfrog_integration
print(f'Performing leapfrog integration with {N} steps...')
for i in tqdm(range(N-1)):
    # Positions at the next time step
    q[i+1,:] = q[i,:] + v[i,:] * dt + 0.5 * a[i,:] * dt2
    # Accelerations of each body in the system at the next time step
    a[i+1,:] = accel_func(q[i+1])        
    # Velocities of each body at the next time step
    v[i+1,:] = v[i,:] + 0.5 * (a[i,:] + a[i+1,:]) * dt
    return q, v
```

## Modern Calculations of Planetary Orbits and Ephemerides

Planetary orbits can be described accurate with just a small number of parameters called **orbital elements**.  These quantities date back to Johannes Kepler and are sometimes referred to as Keplerian elements in his honor.  Here is a picture illustrating the definitions of the orbital elements.

<img src="../graphics/orbital_elements.png">

Citation: By Lasunncty at the English Wikipedia, CC BY-SA 3.0, https://commons.wikimedia.org/w/index.php?curid=8971052

Professionals working in astronomy and space exploration have collaborated over the years and built an excellent infrastructure for efficiently sharing data about the positions and orientations of celestial bodies.  